In [1]:
from numpy import array
from keras.callbacks import CSVLogger
import numpy as np
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import Embedding
import io

Using TensorFlow backend.


In [2]:
# load doc into memory
def load_doc(filename):
    with io.open(filename, 'r', encoding='utf8') as f:
        text = f.read()
    return text

# load
in_filename = r'C:\Users\OFR\Desktop\Thesis work\version four\updated_data_1_1.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

In [3]:
lines

['\ufeffকর্পোরেশনের দোষ',
 'দোষ তো',
 'তো আছে',
 'আছে ।',
 '। কিন্তু',
 'কিন্তু আমরা',
 'আমরা যে',
 'যে সুযোগ',
 'সুযোগ বুঝে',
 'বুঝে হাজার',
 'হাজার হাজার',
 'হাজার কোটি',
 'কোটি টাকার',
 'টাকার ট্যাক্স',
 'ট্যাক্স ফাকি',
 'ফাকি দিয়ে',
 'দিয়ে আসছি',
 'আসছি সেটাও',
 'সেটাও আমরা',
 'আমরা জানি',
 'জানি ।',
 '। এই',
 'এই ভয়ে',
 'ভয়ে সবাই',
 'সবাই স্যু',
 'স্যু করার',
 'করার সাহস',
 'সাহস পাবেনা',
 'পাবেনা ।',
 '। কারণ',
 'কারণ উলটা',
 'উলটা স্যু',
 'স্যু সে',
 'সে নিজে',
 'নিজে খেয়ে',
 'খেয়ে যাবে',
 'যাবে ।',
 '। আপনি',
 'আপনি স্যু',
 'স্যু করে',
 'করে দৃষ্টান্ত',
 'দৃষ্টান্ত শুরু',
 'শুরু করে',
 'করে দিলেই',
 'দিলেই পারেন',
 'পারেন ।',
 '। অন্যরাও',
 'অন্যরাও ধীরে',
 'ধীরে ধীরে',
 'ধীরে উৎসাহিত',
 'উৎসাহিত হবে',
 'হবে ।',
 '। পরিবর্তন',
 'পরিবর্তন আসবে',
 'আসবে ।',
 '। আমাদের',
 'আমাদের সবার',
 'সবার মনেই',
 'মনেই বিদেশে',
 'বিদেশে পড়তে',
 'পড়তে যাওয়ার',
 'যাওয়ার সুপ্ত',
 'সুপ্ত কিন্তু',
 'কিন্তু তীব্র',
 'তীব্র আকাঙ্ক্ষা',
 'আকাঙ্ক্ষা কাজ',
 'কাজ করে',
 'করে ।',
 '। বিদেশে',
 'বিদেশে পড়

In [4]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

In [5]:
vocab_size

22220

In [6]:
sequences

[[22219, 1792],
 [1792, 37],
 [37, 13],
 [13, 1],
 [1, 24],
 [24, 18],
 [18, 21],
 [21, 447],
 [447, 752],
 [752, 185],
 [185, 185],
 [185, 408],
 [408, 753],
 [753, 4624],
 [4624, 5849],
 [5849, 30],
 [30, 1660],
 [1660, 903],
 [903, 18],
 [18, 227],
 [227, 1],
 [1, 4],
 [4, 804],
 [804, 142],
 [142, 2326],
 [2326, 100],
 [100, 1037],
 [1037, 5850],
 [5850, 1],
 [1, 118],
 [118, 5851],
 [5851, 2326],
 [2326, 20],
 [20, 556],
 [556, 400],
 [400, 79],
 [79, 1],
 [1, 39],
 [39, 2326],
 [2326, 3],
 [3, 5852],
 [5852, 85],
 [85, 3],
 [3, 2585],
 [2585, 103],
 [103, 1],
 [1, 4625],
 [4625, 293],
 [293, 293],
 [293, 3905],
 [3905, 12],
 [12, 1],
 [1, 904],
 [904, 362],
 [362, 1],
 [1, 11],
 [11, 353],
 [353, 2327],
 [2327, 990],
 [990, 604],
 [604, 193],
 [193, 3906],
 [3906, 24],
 [24, 1953],
 [1953, 4626],
 [4626, 89],
 [89, 3],
 [3, 1],
 [1, 990],
 [990, 943],
 [943, 241],
 [241, 3334],
 [3334, 582],
 [582, 492],
 [492, 177],
 [177, 825],
 [825, 18],
 [18, 605],
 [605, 168],
 [168, 1096],

In [7]:
length = 2

In [8]:
print(len(sequences))
print(sequences[0][0])

175556
22219


In [9]:
sequences = array(sequences)
sequences

array([list([22219, 1792]), list([1792, 37]), list([37, 13]), ...,
       list([95, 84]), list([84, 454]), list([454, 5848])], dtype=object)

In [10]:
c=1
h=0
while(c!=0):
    limit = sequences.size
    d =[]
    c=0
    for i in range(0 , limit):
        l=len(sequences[i])
        if(l!=length):
            print(sequences[i])
            sequences = np.delete(sequences, i, 0)
            
            c=1
            print("error at ", i)
            print("Deleting :_________________________ ") 
            h = h+1
            break        
print("Total deletion: ", h)     

[13]
error at  6669
Deleting :_________________________ 
[351]
error at  6669
Deleting :_________________________ 
[70]
error at  7723
Deleting :_________________________ 
[6]
error at  7723
Deleting :_________________________ 
[52]
error at  7829
Deleting :_________________________ 
[52]
error at  7829
Deleting :_________________________ 
[2]
error at  7947
Deleting :_________________________ 
[6]
error at  7947
Deleting :_________________________ 
[70]
error at  7954
Deleting :_________________________ 
[52]
error at  7954
Deleting :_________________________ 
[583]
error at  7965
Deleting :_________________________ 
[6]
error at  7965
Deleting :_________________________ 
[70]
error at  7967
Deleting :_________________________ 
[52]
error at  7967
Deleting :_________________________ 
[52]
error at  8008
Deleting :_________________________ 
[52]
error at  8008
Deleting :_________________________ 
[60]
error at  8223
Deleting :_________________________ 
[6277]
error at  8223
Deleting :_

error at  10756
Deleting :_________________________ 
[1108]
error at  10760
Deleting :_________________________ 
[689]
error at  10760
Deleting :_________________________ 
[1721]
error at  10765
Deleting :_________________________ 
[35]
error at  10765
Deleting :_________________________ 
[160]
error at  10790
Deleting :_________________________ 
[734]
error at  10790
Deleting :_________________________ 
[4091]
error at  10814
Deleting :_________________________ 
[3050]
error at  10814
Deleting :_________________________ 
[2]
error at  10857
Deleting :_________________________ 
[1142]
error at  10857
Deleting :_________________________ 
[153]
error at  10863
Deleting :_________________________ 
[10]
error at  10863
Deleting :_________________________ 
[4098]
error at  11304
Deleting :_________________________ 
[243]
error at  11304
Deleting :_________________________ 
[160]
error at  11352
Deleting :_________________________ 
[489]
error at  11352
Deleting :_________________________ 
[

error at  17335
Deleting :_________________________ 
[60]
error at  17339
Deleting :_________________________ 
[1733]
error at  17339
Deleting :_________________________ 
[60]
error at  17343
Deleting :_________________________ 
[43]
error at  17343
Deleting :_________________________ 
[12]
error at  17425
Deleting :_________________________ 
[5]
error at  17425
Deleting :_________________________ 
[12]
error at  17428
Deleting :_________________________ 
[2214]
error at  17428
Deleting :_________________________ 
[60]
error at  18194
Deleting :_________________________ 
[978]
error at  18194
Deleting :_________________________ 
[19]
error at  18199
Deleting :_________________________ 
[8]
error at  18199
Deleting :_________________________ 
[140]
error at  18295
Deleting :_________________________ 
[145]
error at  18295
Deleting :_________________________ 
[1151]
error at  18648
Deleting :_________________________ 
[248]
error at  18648
Deleting :_________________________ 
[108]
error

error at  38880
Deleting :_________________________ 
[742]
error at  38917
Deleting :_________________________ 
[840]
error at  38917
Deleting :_________________________ 
[741]
error at  38933
Deleting :_________________________ 
[275]
error at  38933
Deleting :_________________________ 
[4919]
error at  38954
Deleting :_________________________ 
[1247]
error at  38954
Deleting :_________________________ 
[276]
error at  38960
Deleting :_________________________ 
[1247]
error at  38960
Deleting :_________________________ 
[9]
error at  38968
Deleting :_________________________ 
[17]
error at  38968
Deleting :_________________________ 
[67]
error at  38982
Deleting :_________________________ 
[779]
error at  38982
Deleting :_________________________ 
[599]
error at  39008
Deleting :_________________________ 
[792]
error at  39008
Deleting :_________________________ 
[70]
error at  39018
Deleting :_________________________ 
[473]
error at  39018
Deleting :_________________________ 
[8041

[817]
error at  42985
Deleting :_________________________ 
[650]
error at  42985
Deleting :_________________________ 
[60]
error at  43001
Deleting :_________________________ 
[2010]
error at  43001
Deleting :_________________________ 
[486]
error at  43081
Deleting :_________________________ 
[1255]
error at  43081
Deleting :_________________________ 
[60]
error at  43342
Deleting :_________________________ 
[796]
error at  43342
Deleting :_________________________ 
[6438]
error at  43445
Deleting :_________________________ 
[6]
error at  43445
Deleting :_________________________ 
[1606]
error at  43500
Deleting :_________________________ 
[2210]
error at  43500
Deleting :_________________________ 
[1881]
error at  44053
Deleting :_________________________ 
[1144]
error at  44053
Deleting :_________________________ 
[1879]
error at  44255
Deleting :_________________________ 
[235]
error at  44255
Deleting :_________________________ 
[3678]
error at  44450
Deleting :___________________

...

[11112]
error at  126500
Deleting :_________________________ 
[1449]
error at  126500
Deleting :_________________________ 
[478]
error at  126584
Deleting :_________________________ 
[5400]
error at  126584
Deleting :_________________________ 
[70]
error at  126596
Deleting :_________________________ 
[1406]
error at  126596
Deleting :_________________________ 
[217]
error at  126708
Deleting :_________________________ 
[236]
error at  126708
Deleting :_________________________ 
[214]
error at  126712
Deleting :_________________________ 
[428]
error at  126712
Deleting :_________________________ 
[2836]
error at  127346
Deleting :_________________________ 
[27]
error at  127346
Deleting :_________________________ 
[2]
error at  127599
Deleting :_________________________ 
[22]
error at  127599
Deleting :_________________________ 
[688]
error at  127806
Deleting :_________________________ 
[38]
error at  127806
Deleting :_________________________ 
[2582]
error at  127908
Deleting :______

[512]
error at  134050
Deleting :_________________________ 
[13]
error at  134072
Deleting :_________________________ 
[6]
error at  134072
Deleting :_________________________ 
[70]
error at  134127
Deleting :_________________________ 
[8]
error at  134127
Deleting :_________________________ 
[2274]
error at  134145
Deleting :_________________________ 
[6]
error at  134145
Deleting :_________________________ 
[13]
error at  134310
Deleting :_________________________ 
[11228]
error at  134310
Deleting :_________________________ 
[13]
error at  134312
Deleting :_________________________ 
[2889]
error at  134312
Deleting :_________________________ 
[363]
error at  134320
Deleting :_________________________ 
[11230]
error at  134320
Deleting :_________________________ 
[10989]
error at  134333
Deleting :_________________________ 
[11228]
error at  134333
Deleting :_________________________ 
[4325]
error at  134373
Deleting :_________________________ 
[555]
error at  134373
Deleting :______

[122]
error at  143629
Deleting :_________________________ 
[20658]
error at  143929
Deleting :_________________________ 
[46]
error at  143929
Deleting :_________________________ 
[75]
error at  143940
Deleting :_________________________ 
[51]
error at  143940
Deleting :_________________________ 
[205]
error at  143990
Deleting :_________________________ 
[134]
error at  143990
Deleting :_________________________ 
[1258]
error at  143994
Deleting :_________________________ 
[51]
error at  143994
Deleting :_________________________ 
[7715]
error at  144030
Deleting :_________________________ 
[5]
error at  144030
Deleting :_________________________ 
[188]
error at  144046
Deleting :_________________________ 
[80]
error at  144046
Deleting :_________________________ 
[1952]
error at  144048
Deleting :_________________________ 
[160]
error at  144048
Deleting :_________________________ 
[9702]
error at  144051
Deleting :_________________________ 
[5]
error at  144051
Deleting :__________

[1422]
error at  149681
Deleting :_________________________ 
[20994]
error at  149681
Deleting :_________________________ 
[3258]
error at  149694
Deleting :_________________________ 
[5]
error at  149694
Deleting :_________________________ 
[2]
error at  149717
Deleting :_________________________ 
[5]
error at  149717
Deleting :_________________________ 
[70]
error at  149745
Deleting :_________________________ 
[6]
error at  149745
Deleting :_________________________ 
[405]
error at  149765
Deleting :_________________________ 
[189]
error at  149765
Deleting :_________________________ 
[37]
error at  149806
Deleting :_________________________ 
[334]
error at  149806
Deleting :_________________________ 
[37]
error at  149815
Deleting :_________________________ 
[8]
error at  149815
Deleting :_________________________ 
[37]
error at  149822
Deleting :_________________________ 
[6]
error at  149822
Deleting :_________________________ 
[37]
error at  149827
Deleting :____________________

[3182]
error at  161555
Deleting :_________________________ 
[159]
error at  161555
Deleting :_________________________ 
[31]
error at  161780
Deleting :_________________________ 
[554]
error at  161780
Deleting :_________________________ 
[22]
error at  162361
Deleting :_________________________ 
[2899]
error at  162361
Deleting :_________________________ 
[217]
error at  162366
Deleting :_________________________ 
[3728]
error at  162366
Deleting :_________________________ 
[181]
error at  163893
Deleting :_________________________ 
[269]
error at  163893
Deleting :_________________________ 
[43]
error at  163931
Deleting :_________________________ 
[18]
error at  163931
Deleting :_________________________ 
[13]
error at  170671
Deleting :_________________________ 
[351]
error at  170671
Deleting :_________________________ 
[70]
error at  171725
Deleting :_________________________ 
[6]
error at  171725
Deleting :_________________________ 
[52]
error at  171831
Deleting :_____________

In [11]:
print(sequences[564])
print(sequences.size)

[3912, 132]
173244


In [12]:
sequences = np.array(sequences.tolist())
sequences 

array([[22219,  1792],
       [ 1792,    37],
       [   37,    13],
       ...,
       [   95,    84],
       [   84,   454],
       [  454,  5848]])

In [13]:
sequences.shape

(173244, 2)

In [14]:
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

MemoryError: 

In [ ]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(GRU(100, return_sequences=True))
model.add(GRU(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
csv_logger = CSVLogger('training_1_1.log')
model.fit(X, y, batch_size=500, epochs=1000, callbacks=[csv_logger])

In [ ]:
# save the model to file
model.save(r'C:\Users\OFR\Desktop\Thesis work\version four\model_1_1.h5')

# save the tokenizer
dump(tokenizer, open(r'C:\Users\OFR\Desktop\Thesis work\version four\tokenizer_1_1.pkl', 'wb'))
